# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd 
from pymystem3 import Mystem #для задания 2.4.
m = Mystem()

data = pd.read_csv('/datasets/data.csv') # читаю таблицу для дальнейшей работы с ней. 
data.info() #смотрю общие сведения о таблице.
data.head(10) # смотрю визуализацию таблицы. 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

1 Отрицательные значение в столбце days_employed и большой хвост float - "возможно" округлить
2 Разный формат написания значений - привести к единому формату
3 Столбец total_income сложно читать - приветси в формат int
Также, для дальнейше работы удобно будет классифицировать по значениям education_id, family_status_id, так как значение по id уже определены в таблице. Также, явно необходимо сгруппировать таблицу по столбцу purpose

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data.isna().sum()  #Данным методом вявил, в каких строках есть пропуска. Сумма пропусков одинкова в 2х стобцах, что может говорить о том, что пропуски сделаны в одних и тех же строках.
# print(data[data['total_income'].isna()].head()) #Строки в стобцах days_employed и total_income со значением float, а пропуски с NaN, то есть можно проводить математические операции. Это означает, что пропуски мы можем заменить средним значением столбца. Замена на среднее значение столбца не должно повлиять на результат анализа, так как количетсво заменяемых значений не превышвет 10%.
#Для начала необходимо столбец days_employed избавить от отрицательных значений.
data['days_employed'] = data['days_employed'].apply(abs) #метод abs показался мне наиболее легким и эффективным, нашел здесь https://ru.stackoverflow.com/questions/1133657/Сделать-положительное-число#:~:text=Соединяя%20инверсию%20знака%20с%20созданием,%23%20-5%20print(-abs(b))%20%23%20-2


total_income_mean = data['total_income'].median()  #Для опередения среднего значения столбца, сгруппировал значения по столбцу income_type и ввел дополнительную переменную. 
#print(total_income_meal)
data['total_income'] = data['total_income'].fillna(total_income_mean)

days_employed_mean = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)

dob_mean = data['dob_years'].median() # выявил значание 0 в данном столбце, заменю его на средний возраст
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_mean

print(data.isna().sum()) #повторно проверяю количество пропусков в столбцах, также результат можно проверить методом data.info(), проверить дополнительно не изменился ли фомат данных (были случаи)
#data.info()
data.head(10)




children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Так как данные пропуски были обнаружены в двух стобцах в одинаковых строках, можно предположить, что ошибка была в выгрузке данных. Пропуски NaN, что означает принадлежность к единому формату столбцов. Также, только эти 2 стобца имеют формат float. Что может говорить именно о техничской потери данных при выгрузке дробных чисел. 

Была идея заменить пропуски на средние значения покатегорийно сгруппировав столбцы income_type-total_income и income_type-days_employed. 
Седние значение по группам нашел столбцам нашел - "data.groupby('income_type')['total_income'].mean()", но применить в объеденение не смог. Преположительно через функцию, но расписать ее не хватает времени, чтобы найти решение. Надеюсь сможешь подсказать. 

### Замена типа данных

In [4]:
# Сперва переведу значения flaoat в int для удобной визуализации данных и дальнейшего анализа методом astype 
data['days_employed'] = data['days_employed'].astype('int') 
data['total_income'] = data['total_income'].astype('int')

#столбец education переведу в единый формат. 
data['education'] = data['education'].str.lower()


data.info()
data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**
Все строки переведены в читалбельный формат. 
Видны "странности" в столбце days_employed - крупные значения дней. Но так как заказчик требует аналитику по другим столбцам, оставим эти значения, они нам не помогут в дальнейшем. 


### Обработка дубликатов

In [5]:
#data.info() #смотрю стороки с форматом object - education, family_status, gender,income_type, purpose
# ищем дубликаты
education_counts = data['education'].value_counts() 
print (education_counts)
print ()

data['family_status'] = data['family_status'].str.lower()
family_status_counts = data['family_status'].value_counts() # выровнил по нижнему регистру. 
print (family_status_counts)
print ()

data['gender'] = data['gender'].replace('XNA', 'M')
gender_counts = data['gender'].value_counts() 
#выявил значние 1 XNA, которое нельзя отнести к полу. Одно значение на результат не повлияет, заменю на М, так как данных значений меньше чем F
# 2. Обнуружил нулевое значение. 
print (gender_counts)
print ()

data['income_type']=data['income_type'].replace('студент','пенсионер')
data['income_type']=data['income_type'].replace('безработный','пенсионер')
data['income_type']=data['income_type'].replace('предприниматель','компаньон')
data['income_type']=data['income_type'].replace('в декрете','сотрудник')
income_type_counts = data['income_type'].value_counts() #заменил занчение в количестве 1 - 2 на более подходящую категорb. для чистоты. 
print(income_type_counts)
print ()

purpose_counts = data['purpose'].value_counts() 
print (purpose_counts) # выведу категории в основные категории в следющем задании lemma
print ()

# выявлю количество дубликтов 
#print(data.duplicated().sum()) # Выявил 71 дубликат, необходимо удалить. 
data = data.drop_duplicates().reset_index()
print(data.duplicated().sum()) # проверяю итог



среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F    14236
M     7289
Name: gender, dtype: int64

сотрудник      11120
компаньон       5087
пенсионер       3859
госслужащий     1459
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка

**Вывод**

Дубликаты почищены. Дубликаты выявлял простым методом .value_counts(), так как он максимально прост и нагляден. 

### Лемматизация

In [6]:
#повторно посмотрю на столбец purpose
#purpose_counts = data['purpose'].value_counts() 
#print (purpose_counts)

#главные катеогрии видны: свадьба, жилье, автомобиль, образование. В каждой категрии есть повторяющиеся слова - Свадьба - свадьба, Жилья - недвижимость, Автомобиль - автомобиль, Образование - образование.
#с данными категориями и повторяющимися словами и буду работать. Для этого сделаю функцию с методом lemma для группировки категорий.

lemmas = m.lemmatize(''.join(data['purpose'].unique()))
from collections import Counter
print (Counter(lemmas)) #благодаря Counter полчилось вывести наиболее повторяемы слова в столбце purpose. Данным способом получил наименование категорий. 

def lemmatizer (data):
    for i in range(0, len(data['purpose'])): #для изменения списока в цикле
        lemmas = m.lemmatize(data.loc[i,'purpose'])
        if 'свадьба' in lemmas:
            data.loc[i,'purpose'] = 'свадьба'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            data.loc[i,'purpose'] = 'недвижимость'
        if 'автомобиль' in lemmas:
            data.loc[i,'purpose'] = 'автомобиль'
        if 'образование' in lemmas:
            data.loc[i,'purpose'] = 'образование'
    return data

data = lemmatizer(data)
#print (data.head(10)) #проверил реудьтат и чтобы ничего не "поехало". 

Counter({' ': 59, 'с': 5, 'покупка': 4, 'свой': 3, 'жилье': 2, 'для': 2, 'недвижимостипокупка': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'высокий': 2, 'жильяприобретение': 1, 'автомобилядополнительный': 1, 'образованиесыграть': 1, 'свадьбуоперация': 1, 'жильемобразованиен': 1, 'проведение': 1, 'свадьбыпокупка': 1, 'семьипокупка': 1, 'недвижимостистроительство': 1, 'собственный': 1, 'недвижимостинедвижимостьстроительство': 1, 'недвижимостина': 1, 'автомобиляна': 1, 'автомобиляоперация': 1, 'недвижимостьюстроительство': 1, 'недвижимостижильеоперация': 1, 'со': 1, 'недвижимостьюавтомобилизаняться': 1, 'образованиемсделка': 1, 'автомобилемполучение': 1, 'образованияавтомобильсвадьбаполучение': 1, 'дополнительный': 1, 'образованияпокупка': 1, 'жильяоперация': 1, 'недвижимостьюполучение': 1, 'образованиясва': 1, 'автомобильсделка': 1, 'автомобилемпрофильный': 1, 'образованиевысший': 1, 'образованиепокупка': 1, 'сдачина': 1, 'автомобиляремонт': 1, 'жильюзаняться': 1, 'образование': 1

**Вывод**
Благодаря Lemma в данных осталось только 4 категории из множетсва, что поможет анализировать данные для выполнения задания заказчика. 

### Категоризация данных

In [8]:
#сперва посмотрю уникальные значения столбеца 
#print(data['children'].unique())  

# КАТЕГОРИЯ ДЛЯ ДЕТЕЙ 
children_group = data[['children']] #ввел индексацию по 2 столбцам
no_children = children_group [children_group['children'] == 0] # определил категории 
one_children = children_group[children_group['children'] == 1]
more_children = children_group[children_group['children'] > 1]

print('Заемщики без детей') #проверяю категоризацию
print (no_children.count())
print()
print('Заемщики с одним ребенком')
print(one_children.count())
print()
print('Многодетные заемщики')
print(more_children.count())
print()

#КАТЕГОРИЯ ПО ДОХОДУ
income_group = data[['total_income']] 
income_poor = income_group[income_group['total_income'] <= 50000]
income_low = income_group[(income_group['total_income'] >= 50001) & (income_group['total_income'])]
income_middle = income_group[(income_group['total_income'] >= 150001) & (income_group['total_income'] <= 500000)]
income_high = income_group[income_group['total_income'] >= 500001]
                                                                     
print('Заемщики с низким доходом') 
print (income_poor.count())
print()
print('Заемщики с доходом выше низкого')
print(income_low.count())
print()
print('Заемщики со средним доходом')
print(income_middle.count())
print()
print('Заемщики с высоким доходом')
print(income_high.count())
print()

#КАТЕГОРИЯ ПО ЦЕЛЯМ
purpose_group = data[['purpose']] 
wedding = purpose_group[purpose_group['purpose'] == 'свадьба']
housing = purpose_group[purpose_group['purpose'] == 'недвижимость']
car = purpose_group[purpose_group['purpose'] == 'автомобиль']
education = purpose_group[purpose_group['purpose'] == 'образование']
                                                                     
print('Цель заема - свадьба') 
print (wedding.count())
print()
print('Цель заема - недвижимость')
print(housing.count())
print()
print('Цель заема - автомобиль')
print(car.count())
print()
print('Цель заема - образование')
print(education.count())
print()

Заемщики без детей
children    14090
dtype: int64

Заемщики с одним ребенком
children    4808
dtype: int64

Многодетные заемщики
children    2508
dtype: int64

Заемщики с низким доходом
total_income    372
dtype: int64

Заемщики с доходом выше низкого
total_income    11556
dtype: int64

Заемщики со средним доходом
total_income    8962
dtype: int64

Заемщики с высоким доходом
total_income    222
dtype: int64

Цель заема - свадьба
purpose    2323
dtype: int64

Цель заема - недвижимость
purpose    10811
dtype: int64

Цель заема - автомобиль
purpose    4306
dtype: int64

Цель заема - образование
purpose    4013
dtype: int64



**Вывод**
Согласно комментариям выделил категории замещиков по количеству детей, доходу и целей. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
children_debt = (data[data['debt'] == 1].groupby('debt')['children'].sum())
no_children_debt = children_debt.sum()/no_children.count()
one_children_debt = children_debt.sum()/one_children.count()
more_children_debt = children_debt.sum()/more_children.count()
print ('Процент возврата у бездетных заемщиков')
print (no_children_debt*100)
print()
print ('Процент возврата у заемщиков одним ребенком')
print (one_children_debt*100)
print()
print ('Процент возврата у многодетных заемщиков')
print (more_children_debt*100)
print()


Процент возврата у бездетных заемщиков
children    7.721789
dtype: float64

Процент возврата у заемщиков одним ребенком
children    22.628952
dtype: float64

Процент возврата у многодетных заемщиков
children    43.38118
dtype: float64



**Вывод**

Заемщики не имещие детей реже возвращают кредиты. Заемщики с одним и более детьми возвращают чаще. 
Ответ - зависимость оплаты кредита от наличия ребенка есть. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
#print(data['family_status'].value_counts()) 

family_status_debt = (data[data['debt'] == 1].groupby('family_status')['debt'].count() / data.groupby('family_status')['debt'].count())
print ((family_status_debt)*100)

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.349398
женат / замужем          7.545182
не женат / не замужем    9.750890
Name: debt, dtype: float64


**Вывод**

Не женатые и незамужние оплчивает кредиты чаще чем все остальные категории. Возможно ошибка в данных или расчетах. Так как, по логике, платить тдолжны семейные люди, но они на 2м месте. 

Ответ - зависимость есть. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
income_debt = (data[data['debt'] == 1].groupby('debt')['total_income'].sum())
income_poor_debt = income_debt.sum()/income_poor.count()
income_low_debt = income_debt.sum()/income_low.count()
income_middle_debt = income_debt.sum()/income_middle.count()
income_high_debt = income_debt.sum()/income_high.count()
print ('Процент возврата у заемщиков с низким доходом')
print (income_poor_debt*100)
print()
print ('Процент возврата у заемщиков с доходом выше среднего')
print (income_low_debt*100)
print()
print ('Процент возврата у заемщиков со средним доходом')
print (income_middle_debt*100)
print()
print ('Процент возврата у заемщиков с высоким доходом')
print (income_high_debt*100)
print()

Процент возврата у заемщиков с низким доходом
total_income    7.550635e+07
dtype: float64

Процент возврата у заемщиков с доходом выше среднего
total_income    2.430630e+06
dtype: float64

Процент возврата у заемщиков со средним доходом
total_income    3.134162e+06
dtype: float64

Процент возврата у заемщиков с высоким доходом
total_income    1.265241e+08
dtype: float64



**Вывод**
По результатам видим, что люди с низким доходом чаще выплачивают кредит во время. Это говорит о нежелании погрязнуть в долгах, не имея сверх дохода. С очень низким доходом и с высоким доходом, заемщики выплачивают реже кредит, так как у одних денег нет, другие инвестируют свободные деньги. 

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
purpose_debt = (data[data['debt'] == 1].groupby('purpose')['debt'].count() / data.groupby('purpose')['debt'].count())

print(purpose_debt.sort_values()*100)


purpose
недвижимость    7.233373
свадьба         8.006888
образование     9.220035
автомобиль      9.359034
Name: debt, dtype: float64


**Вывод**

Лидирует недвижимость в целях по кредиту и, удивительно, - свадьба! 

Зависимость есть. 

In [32]:
# Код ревьюера
purpose_debt = purpose_debt.reset_index().sort_values(by='debt', ascending=False)
purpose_debt['debt'] = purpose_debt['debt'].map('{:.1%}'.format)
purpose_debt

,index,purpose,debt
0,0,автомобиль,9.4%
1,2,образование,9.2%
2,3,свадьба,8.0%
3,1,недвижимость,7.2%


## Шаг 4. Общий вывод

Проанализировав данные можено сделать следующий вывод: 

1. Наличие детей влияет в положительную сторону на выплаты по кредиту
2. Исходя из предыдущего пункта логичны показатели по целям кредита - заем на недвижимость для семьи
3. Семейное положение, в масштабах, не влияет на выплаты по кредиту 
4. Выгоднее выдавать кредиты людам с достатком от 50000 до 150000, так как их платежеспособность лучше. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.